In [ ]:
# https://python.langchain.com/docs/get_started/quickstart

In [1]:
# ----- 環境変数読み込み -----
from dotenv import load_dotenv

load_dotenv()

# ----- Chat準備 -----
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI()

In [2]:
# ----- LLMの最もシンプルな使い方 -----
from langchain.schema import HumanMessage
text = "日本で一番高い山は?"
# HumanMessageは、ユーザのインプットと思えばOK
messages = [HumanMessage(content=text)]

# chat実行
chat_model.predict_messages(messages)

AIMessage(content='日本で一番高い山は富士山です。富士山は標高3,776メートルで、日本の象徴的な山として知られています。', additional_kwargs={}, example=False)

In [6]:
# ----- プロンプトの使い方（単一プロンプト） -----
from langchain.prompts import PromptTemplate
# PromptTemplateのformatを使うと、プロンプトのテンプレートに変数を埋め込める
prompt = PromptTemplate.from_template("{app}はどの会社が開発しているアプリケーションですか?")
message = prompt.format(app="LINE")
chat_model.predict(message)

'LINEは韓国のNHN株式会社（現在のNAVER株式会社）が開発しています。'

In [7]:
# ----- プロンプトの使い方（複数プロンプト） -----
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
# システムインプット用のテンプレートを定義
system_message_template = "与えられたコードを{lang_from}から{lang_to}に書き換えてください。"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)
# ユーザインプット用のテンプレートを定義
human_message_template = "{code}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)
# Chatで使用するプロンプトを定義＆変数埋め込み（Webではこれ以降をリクエストごとに実行）
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_message = chat_prompt.format_messages(lang_from="Java", lang_to="Python", code="System.out.println(\"Hello\");")
# chat実行
chat_model.predict_messages(chat_message)

AIMessage(content='print("Hello")', additional_kwargs={}, example=False)

In [20]:
# ----- outputの調整 -----
# cf. https://python.langchain.com/docs/get_started/quickstart#output-parsers
# サンプル：chatからの返答をCSVで受け取る

from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # インプット時にCSV形式で出力するように指示する
    template="List tree of {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

from langchain.llms import OpenAI
model = OpenAI(temperature=0)
_input = prompt.format(subject="most usable develop language")
print(f'_input: ${_input}')
output = model(_input)
print(f'output: ${output}')

# CSV形式の出力を、python上でリストとして扱えるように変換する
output_parser.parse(output)

_input: $List tree of most usable develop language.
Your response should be a list of comma separated values, eg: `foo, bar, baz`
output: $

Java, JavaScript, Python, C, C++, C#, PHP, SQL, HTML, CSS


['Java', 'JavaScript', 'Python', 'C', 'C++', 'C#', 'PHP', 'SQL', 'HTML', 'CSS']